In [1]:
# --- STEP 1: INSTALL & SETUP ---
!pip install tensorflow ultralytics plotly -q

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# --- STEP 2: DATA PREP (With Advanced Augmentation) ---
# Ensure your dataset folder 'dataset' has subfolders: buffalo, elephant, rhino, zebra
base_dir = 'dataset'
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,       # Rotates images (aerial views often rotate)
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,          # Zoom in/out
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# --- STEP 3: TRANSFER LEARNING (MobileNetV2) ---
# We download the brain of a model trained on millions of images
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# We freeze the base so we don't ruin its pre-learned knowledge
base_model.trainable = False

# Add our own "Wildlife Head" to the model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)  # Prevents overfitting
predictions = Dense(4, activation='softmax')(x) # 4 classes: Buffalo, Elephant, Rhino, Zebra

model = Model(inputs=base_model.input, outputs=predictions)

# Compile with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# --- STEP 4: TRAINING ---
history = model.fit(
    train_generator,
    epochs=15,  # Transfer learning learns faster!
    validation_data=validation_generator
)

# Save the smarter model
model.save('wildlife_v2_mobilenet.h5')
print("High-Accuracy Model Saved!")

C:\Users\admin\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.0 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\admin\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.0 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\admin\AppData\Roaming\Python\Python313\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.0 at tensorflow/core/framework/resource_handle.proto. Please 

Found 1204 images belonging to 4 classes.
Found 300 images belonging to 4 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 103s 3s/step - accuracy: 0.4377 - loss: 1.3770 - val_accuracy: 0.7067 - val_loss: 0.8264
Epoch 2/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.6786 - loss: 0.8288 - val_accuracy: 0.8300 - val_loss: 0.5192
Epoch 3/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.7733 - loss: 0.5926 - val_accuracy: 0.8467 - val_loss: 0.4048
Epoch 4/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.8173 - loss: 0.4854 - val_accuracy: 0.8767 - val_loss: 0.3602
Epoch 5/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 74s 2s/step - accuracy: 0.8472 - loss: 0.4311 - val_accuracy: 0.8633 - val_loss: 0.3561
Epoch 6/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.8522 - loss: 0.3809 - val_accuracy: 0.9100 - val_loss: 0.2612
Epoch 7/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - accuracy: 0.8663 - loss: 0.3601 - val_accuracy: 0.9100 - va

High-Accuracy Model Saved!
